<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-07 14:25:25
-------------------
qualified stocks: 97
with latest results: 90
still star stocks: 48
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.23 C
-------------------
Today PnL: 38.28 K (0.31%)
Current PnL: -21.70 L (-15.39%)
CY Booked + Current PnL: -19.69 L (-13.96%)
-------------------
Total profit:  3.67 L
Total loss:  -25.37 L
-------------------
Total Booked + Current PnL: 7.29 L (6.4%)
Total Booked PnL: 28.99 L (25.42%)
Curr Year Booked PnL: 2.01 L (1.64%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.06 C
Est FTT PnL: 82.58 L (67.11%)
-------------------
Est LTT:  2.67 C
Est LTT PnL: 1.44 C (116.75%)
Deployed:  1.14 C
Current:  1.23 C
CAGR/XIRR %: 5.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HDFCBANK,1561.04,2170.00,4.17,X-LC,76.29,102735.0,20000.0,12277.0,0.35,24.17,11.95,39.01,186.0,1.63,0.85,35.74,X40,BTT,BANKS
73,SBILIFE,1496.49,1924.99,-1.29,H-LC,90.72,146597.0,20892.0,15099.0,1.11,16.62,10.30,28.63,152.0,1.38,1.21,30.60,AR,ATH,INSURANCE
60,PIDILITIND,2504.06,3576.00,-14.47,X-LC,40.21,89187.0,14065.0,18096.0,-1.06,18.72,20.29,42.81,57.0,0.78,0.74,11.85,X40,BTT,CHEMICALS
55,MRF,113820.50,150504.14,21.99,X-MC,98.97,280460.0,52819.0,20558.0,3.88,23.20,7.33,32.23,214.0,2.57,2.32,36.60,AR,ATH,TYRES
35,HDFCLIFE,562.05,897.00,-5.23,H-LC,65.98,90338.0,20082.0,21790.0,0.01,28.58,24.12,59.59,188.0,0.92,0.75,35.68,X40,BTT,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,20.82,M-SC,100.00,76344.0,-4992.0,77130.0,5.02,-6.14,101.03,88.69,155.0,-0.06,0.63,28.74,XR,NTT,CHEMICALS
93,VIPIND,488.80,718.00,-754.43,H-SC,95.88,62302.0,-33014.0,77709.0,4.74,-34.64,124.73,46.89,68.0,-0.42,0.52,22.88,X40N,NTT,MISC
84,TATAMOTORS,798.82,1065.00,-55.10,M-LC,86.60,138949.0,-24809.0,79382.0,4.57,-15.15,57.13,33.32,2.0,-0.31,1.15,16.91,XY24,NTT,AUTO
55,MRF,113820.50,150504.14,21.99,X-MC,98.97,280460.0,52819.0,20558.0,3.88,23.20,7.33,32.23,214.0,2.57,2.32,36.60,AR,ATH,TYRES
95,WHIRLPOOL,1112.81,2270.00,-38.19,M-SC,88.66,110295.0,10142.0,94004.0,2.42,10.13,85.23,103.99,27.0,0.11,0.91,32.96,X40,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANPAINT,2961.56,4250.0,-30.32,X-LC,17.53,159759.0,-41627.0,129245.0,-2.84,-20.67,80.90,43.51,59.0,-0.32,1.32,10.19,X40,BTT,PAINTS
30,GREENPANEL,375.16,537.0,148.71,M-SC,39.18,111044.0,-72034.0,151009.0,-2.69,-39.35,135.99,43.14,60.0,-0.48,0.92,4.45,XY24,NTT,MISC
20,COFFEEDAY,59.14,80.0,-59.76,L-SC,60.82,59021.0,-54528.0,94581.0,-2.01,-48.02,160.25,35.27,159.0,-0.58,0.49,44.45,XR,NTT,HOTELS
41,INDUSINDBK,1354.50,1800.0,-35.58,H-MC,96.91,47413.0,-29794.0,55189.0,-1.76,-38.59,116.40,32.89,84.0,-0.54,0.39,30.57,XR,NTT,BANKS
63,RAJESHEXPO,517.65,583.0,1691.67,M-SC,24.74,48869.0,-88308.0,105625.0,-1.76,-64.38,216.14,12.62,154.0,-0.84,0.40,21.54,X40N,NTT,JEWELLERY


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TANLA,991.56,1963.11,-58.95,H-SC,73.20,104202.0,-114933.0,329643.0,-0.73,-52.45,316.35,97.98,1.0,-0.35,0.86,12.87,AR,ATH,IT
84,TATAMOTORS,798.82,1065.00,-55.10,M-LC,86.60,138949.0,-24809.0,79382.0,4.57,-15.15,57.13,33.32,2.0,-0.31,1.15,16.91,XY24,NTT,AUTO
10,AWL,361.39,485.00,-69.33,H-MC,9.28,135084.0,-51032.0,114686.0,1.47,-27.42,84.90,34.20,3.0,-0.44,1.12,11.26,XY24,NTT,FMCG
2,ABB,5319.37,8953.12,-40.65,H-LC,68.04,134650.0,1666.0,89179.0,0.97,1.25,66.23,68.31,4.0,0.02,1.11,9.80,AR,ATH,ELECTRICAL
94,VOLTAS,1272.00,1929.20,-20.53,H-MC,8.25,135256.0,-4664.0,76961.0,-0.86,-3.33,56.90,51.67,5.0,-0.06,1.12,2.16,AR,ATH,AC


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MRF,113820.50,150504.14,21.99,X-MC,98.97,280460.0,52819.0,20558.0,3.88,23.20,7.33,32.23,214.0,2.57,2.32,36.60,AR,ATH,TYRES
35,HDFCLIFE,562.05,897.00,-5.23,H-LC,65.98,90338.0,20082.0,21790.0,0.01,28.58,24.12,59.59,188.0,0.92,0.75,35.68,X40,BTT,INSURANCE
34,HDFCBANK,1561.04,2170.00,4.17,X-LC,76.29,102735.0,20000.0,12277.0,0.35,24.17,11.95,39.01,186.0,1.63,0.85,35.74,X40,BTT,BANKS
28,GICRE,340.22,583.00,1.27,H-MC,56.70,96548.0,16937.0,39874.0,1.00,21.27,41.30,71.36,182.0,0.42,0.80,31.86,XY24,BTT,INSURANCE
4,ADANIPORTS,1103.69,1583.00,3.60,M-LC,93.81,182785.0,31579.0,34089.0,1.01,20.89,18.65,43.43,81.0,0.93,1.51,26.70,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,NHPC,82.08,115.00,194.74,M-MC,20.62,142938.0,1925.0,54631.0,1.82,1.36,38.22,40.11,196.0,0.04,1.18,16.85,XY24,NTT,POWER
61,POLYCAB,5005.92,7541.35,16.07,H-MC,89.69,164934.0,24768.0,46231.0,-0.26,17.67,28.03,50.65,119.0,0.54,1.36,25.88,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,3.60,M-LC,93.81,182785.0,31579.0,34089.0,1.01,20.89,18.65,43.43,81.0,0.93,1.51,26.70,XY24,NTT,MISC
28,GICRE,340.22,583.00,1.27,H-MC,56.70,96548.0,16937.0,39874.0,1.00,21.27,41.30,71.36,182.0,0.42,0.80,31.86,XY24,BTT,INSURANCE
6,ANGELONE,2328.67,3033.00,-6.93,H-SC,71.13,151723.0,359.0,45426.0,1.52,0.24,29.94,30.25,115.0,0.01,1.25,20.08,X40N,NTT,FINANCE


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
96,WIPRO,243.46,420.00,-11.98,M-LC,16.49,150840.0,-105.0,109555.0,0.87,-0.07,72.63,72.51,45.0,-0.00,1.25,13.57,XR,NTT,IT
18,CAMS,3643.00,5250.99,-8.10,H-SC,28.87,101581.0,-423.0,45447.0,0.55,-0.41,44.74,44.14,14.0,-0.01,0.84,18.95,XR,ATH,MISC
1,AARTIIND,487.04,919.00,20.82,M-SC,100.00,76344.0,-4992.0,77130.0,5.02,-6.14,101.03,88.69,155.0,-0.06,0.63,28.74,XR,NTT,CHEMICALS
94,VOLTAS,1272.00,1929.20,-20.53,H-MC,8.25,135256.0,-4664.0,76961.0,-0.86,-3.33,56.90,51.67,5.0,-0.06,1.12,2.16,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-29.61,M-MC,29.90,123803.0,-5349.0,92419.0,-0.26,-4.14,74.65,67.42,74.0,-0.06,1.02,20.41,XR,NTT,BANKS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HDFCLIFE,562.05,897.00,-5.23,H-LC,65.98,90338.0,20082.0,21790.0,0.01,28.58,24.12,59.59,188.0,0.92,0.75,35.68,X40,BTT,INSURANCE
34,HDFCBANK,1561.04,2170.00,4.17,X-LC,76.29,102735.0,20000.0,12277.0,0.35,24.17,11.95,39.01,186.0,1.63,0.85,35.74,X40,BTT,BANKS
89,UJJIVANSFB,52.77,60.00,-42.86,M-SC,83.51,110538.0,-31941.0,51466.0,-0.07,-22.42,46.56,13.70,174.0,-0.62,0.91,30.47,X40N,NTT,BANKS
73,SBILIFE,1496.49,1924.99,-1.29,H-LC,90.72,146597.0,20892.0,15099.0,1.11,16.62,10.30,28.63,152.0,1.38,1.21,30.60,AR,ATH,INSURANCE
54,MEDANTA,1022.60,1486.00,3.67,X-MC,18.56,182750.0,27315.0,43129.0,1.25,17.57,23.60,45.32,170.0,0.63,1.51,24.35,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,20.82,M-SC,100.00,76344.0,-4992.0,77130.0,5.02,-6.14,101.03,88.69,155.0,-0.06,0.63,28.74,XR,NTT,CHEMICALS
55,MRF,113820.50,150504.14,21.99,X-MC,98.97,280460.0,52819.0,20558.0,3.88,23.20,7.33,32.23,214.0,2.57,2.32,36.60,AR,ATH,TYRES
79,SONACOMS,578.05,1006.00,-31.35,M-MC,97.94,89162.0,-11997.0,86888.0,1.40,-11.86,97.45,74.03,25.0,-0.14,0.74,25.31,AR,BTT,AUTO
41,INDUSINDBK,1354.50,1800.00,-35.58,H-MC,96.91,47413.0,-29794.0,55189.0,-1.76,-38.59,116.40,32.89,84.0,-0.54,0.39,30.57,XR,NTT,BANKS
93,VIPIND,488.80,718.00,-754.43,H-SC,95.88,62302.0,-33014.0,77709.0,4.74,-34.64,124.73,46.89,68.0,-0.42,0.52,22.88,X40N,NTT,MISC


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.91
1,25,41.46
2,50,68.45


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.40
X40     14.84
X40N    13.47
AR      12.81
XY25    11.08
XR      10.10
X200     1.13
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.52
X-LC    19.10
M-SC    15.68
X-MC     8.01
M-LC     7.84
M-MC     7.45
H-MC     7.03
H-LC     6.16
X-SC     2.50
L-SC     2.20
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,11.71,-12.05,66.43
FMCG,10.83,-2.94,34.45
IT,8.78,-28.32,95.49
FINANCE,7.13,-28.38,86.31
INSURANCE,6.89,-1.94,39.11
MISC,6.26,-36.23,85.52
PAINTS,5.64,-20.58,56.27
ELECTRICAL,4.10,-11.77,68.73
REFINERIES,3.70,8.91,7.88


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3281503.0,29
XR,1321131.0,15
X40N,1269890.0,17
AR,1095696.0,13
X40,804263.0,14
SR,208781.0,2
XY25,196474.0,6
X200,80744.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2368161.0,21
M-SC,2022104.0,20
X-LC,786132.0,13
M-MC,689851.0,7
H-MC,408455.0,7
X-SC,373657.0,4
M-LC,368570.0,6
X-MC,336483.0,6
L-SC,332236.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      980617.0      6
M-SC       XY24      823329.0      6
H-SC       X40N      590975.0      7
M-MC       XY24      485387.0      4
M-SC       XR        451787.0      5
X-LC       X40       406586.0      6
M-SC       X40N      398835.0      5
H-SC       AR        393077.0      2
           XR        313405.0      4
M-LC       XY24      235024.0      4
H-MC       XY24      196713.0      3
X-LC       AR        196089.0      3
M-SC       X40       167876.0      2
L-SC       XR        166981.0      2
H-LC       AR        157401.0      3
X-MC       XY24      141047.0      2
X-SC       XR        131795.0      1
           SR        123468.0      1
           X40N      118394.0      2
X-MC       X40N      115455.0      2
M-LC       XR        109555.0      1
X-LC       XY25      102713.0      3
L-SC       XY24       95497.0      1
M-SC       AR         94964.0      1
M-MC       XR         92419.0      1
H-SC       X40        90087.0      2
M-MC       AR         86888.0      1
M-SC       SR         85313.0      1
X-LC       X200       80744.0      1
H-MC       AR         76961.0      1
L-SC       AR         69758.0      1
X-MC       X40        59423.0      1
H-MC       XR         55189.0      1
H-LC       X40        46930.0      2
H-MC       X40N       46231.0      1
H-LC       XY25       44613.0      1
H-MC       X40        33361.0      1
M-MC       XY25       25157.0      1
M-LC       XY25       23991.0      1
X-MC       AR         20558.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
